In [1]:
import pandas as pd
import numpy as np
import yaml
from tqdm import tqdm
from pathlib import Path
from glob import glob
from functools import partial
from multiprocessing import Pool

import shapefile
import shapely
from shapely.geometry import Point

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from projections.shapefiles import load_shapes_by_country
from projections.models import Records
from projections import raster, utils

pd.set_option('max_columns', None)

In [2]:
output_folder = Path('../Output/Precipitaciones/Raster/SCPDSI/')

In [3]:
df = pd.read_csv('../Data/Precipitaciones/scpdsi_80-20.csv')
print(df.shape)
print('Unique locs', df.drop_duplicates(['lon', 'lat']).shape)
df.head()

(64040, 494)
Unique locs (64040, 494)


,lon,lat,80-01,80-02,80-03,80-04,80-05,80-06,80-07,80-08,80-09,80-10,80-11,80-12,81-01,81-02,81-03,81-04,81-05,81-06,81-07,81-08,81-09,81-10,81-11,81-12,82-01,82-02,82-03,82-04,82-05,82-06,82-07,82-08,82-09,82-10,82-11,82-12,83-01,83-02,83-03,83-04,83-05,83-06,83-07,83-08,83-09,83-10,83-11,83-12,84-01,84-02,84-03,84-04,84-05,84-06,84-07,84-08,84-09,84-10,84-11,84-12,85-01,85-02,85-03,85-04,85-05,85-06,85-07,85-08,85-09,85-10,85-11,85-12,86-01,86-02,86-03,86-04,86-05,86-06,86-07,86-08,86-09,86-10,86-11,86-12,87-01,87-02,87-03,87-04,87-05,87-06,87-07,87-08,87-09,87-10,87-11,87-12,88-01,88-02,88-03,88-04,88-05,88-06,88-07,88-08,88-09,88-10,88-11,88-12,89-01,89-02,89-03,89-04,89-05,89-06,89-07,89-08,89-09,89-10,89-11,89-12,90-01,90-02,90-03,90-04,90-05,90-06,90-07,90-08,90-09,90-10,90-11,90-12,91-01,91-02,91-03,91-04,91-05,91-06,91-07,91-08,91-09,91-10,91-11,91-12,92-01,92-02,92-03,92-04,92-05,92-06,92-07,92-08,92-09,92-10,92-11,92-12,93-01,93-02,93-03,93-04,93-05,93-06,93-07,93-08,93-09,93-10,93-11,93-12,94-01,94-02,94-03,94-04,94-05,94-06,94-07,94-08,94-09,94-10,94-11,94-12,95-01,95-02,95-03,95-04,95-05,95-06,95-07,95-08,95-09,95-10,95-11,95-12,96-01,96-02,96-03,96-04,96-05,96-06,96-07,96-08,96-09,96-10,96-11,96-12,97-01,97-02,97-03,97-04,97-05,97-06,97-07,97-08,97-09,97-10,97-11,97-12,98-01,98-02,98-03,98-04,98-05,98-06,98-07,98-08,98-09,98-10,98-11,98-12,99-01,99-02,99-03,99-04,99-05,99-06,99-07,99-08,99-09,99-10,99-11,99-12,00-01,00-02,00-03,00-04,00-05,00-06,00-07,00-08,00-09,00-10,00-11,00-12,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,01-11,01-12,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,03-11,03-12,04-01,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11,04-12,05-01,05-02,05-03,05-04,05-05,05-06,05-07,05-08,05-09,05-10,05-11,05-12,06-01,06-02,06-03,06-04,06-05,06-06,06-07,06-08,06-09,06-10,06-11,06-12,07-01,07-02,07-03,07-04,07-05,07-06,07-07,07-08,07-09,07-10,07-11,07-12,08-01,08-02,08-03,08-04,08-05,08-06,08-07,08-08,08-09,08-10,08-11,08-12,09-01,09-02,09-03,09-04,09-05,09-06,09-07,09-08,09-09,09-10,09-11,09-12,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,10-11,10-12,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,11-09,11-10,11-11,11-12,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09,12-10,12-11,12-12,13-01,13-02,13-03,13-04,13-05,13-06,13-07,13-08,13-09,13-10,13-11,13-12,14-01,14-02,14-03,14-04,14-05,14-06,14-07,14-08,14-09,14-10,14-11,14-12,15-01,15-02,15-03,15-04,15-05,15-06,15-07,15-08,15-09,15-10,15-11,15-12,16-01,16-02,16-03,16-04,16-05,16-06,16-07,16-08,16-09,16-10,16-11,16-12,17-01,17-02,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,18-01,18-02,18-03,18-04,18-05,18-06,18-07,18-08,18-09,18-10,18-11,18-12,19-01,19-02,19-03,19-04,19-05,19-06,19-07,19-08,19-09,19-10,19-11,19-12,20-01,20-02,20-03,20-04,20-05,20-06,20-07,20-08,20-09,20-10,20-11,20-12
0,-72.25,83.25,-0.46,-0.39,-0.34,-0.34,-0.99,-1.07,-1.41,2.14,4.22,3.89,3.35,2.89,2.49,2.14,1.85,1.58,2.06,2.86,3.46,2.37,2.60,2.53,2.18,1.88,1.62,1.40,1.20,1.31,1.73,2.45,2.67,-0.24,-0.25,-0.44,-0.38,-0.33,-0.28,-0.24,-0.21,-0.20,-0.32,0.44,4.26,-0.51,-0.46,-0.61,-0.53,-0.45,-0.39,-0.34,-0.29,-0.34,-0.44,-0.64,1.55,1.03,1.75,2.70,2.32,2.00,1.73,1.49,1.28,1.09,1.63,2.07,1.82,-0.41,-1.17,-0.92,-0.85,-0.79,-0.73,-0.68,-0.63,-0.67,-1.27,-1.37,1.40,-1.00,-0.99,-0.64,-0.55,-0.47,-0.41,-0.35,-0.30,-0.28,-0.17,-0.40,2.17,-0.27,-1.08,-0.56,-0.48,-0.42,-0.36,-0.31,-0.27,-0.26,-1.06,0.40,-0.27,-0.40,-0.34,-0.27,-0.23,-0.20,-0.17,-0.15,-0.13,-0.17,0.57,0.58,1.02,1.97,1.73,1.77,1.52,1.31,1.13,0.97,0.84,1.68,2.03,-0.42,0.45,-1.05,-1.11,-1.33,0.00,0.00,0.00,0.00,0.00,-0.09,-0.41,-0.44,0.44,0.68,1.29,1.60,1.38,1.19,1.02,0.88,0.76,0.80,1.29,2.69,4.39,3.98,-0.47,-0.19,-0.16,-0.14,-0.12,-0.10,-0.09,-0.04,-0.29,-0.56,-0.13,-0.52,-0.81,-1.00,-0.86,-0.74,-0.64,-0.55,-0.48,-0.44,-1.12,-1.42,-1.90,1.66,-0.88,-0.69,-0.59,-0.51,-0.44,-0.38,-0.33,-0

# ADM

In [4]:
cru = pd.read_csv(output_folder / '../CRU/loc_map.csv')
cru.head()

,index,lat,lon,adm0,adm1,adm2,intersection_ratio
0,14876,4.25,73.25,MDV,NaN,NaN,0.000956
1,14877,4.25,73.75,MDV,NaN,NaN,0.003977
2,29703,34.25,35.75,LBN,LBN.1_1,LBN.1.1_1,0.003222
3,29704,34.25,36.25,LBN,LBN.1_1,LBN.1.1_1,0.072300
4,29704,34.25,36.25,LBN,LBN.2_1,LBN.2.1_1,0.544474


In [5]:
# CRU and SCPDSI share the same locations
df = df.merge(cru, on=['lon', 'lat'])
print(df.shape)
df.head()

0


,lon,lat,80-01,80-02,80-03,80-04,80-05,80-06,80-07,80-08,80-09,80-10,80-11,80-12,81-01,81-02,81-03,81-04,81-05,81-06,81-07,81-08,81-09,81-10,81-11,81-12,82-01,82-02,82-03,82-04,82-05,82-06,82-07,82-08,82-09,82-10,82-11,82-12,83-01,83-02,83-03,83-04,83-05,83-06,83-07,83-08,83-09,83-10,83-11,83-12,84-01,84-02,84-03,84-04,84-05,84-06,84-07,84-08,84-09,84-10,84-11,84-12,85-01,85-02,85-03,85-04,85-05,85-06,85-07,85-08,85-09,85-10,85-11,85-12,86-01,86-02,86-03,86-04,86-05,86-06,86-07,86-08,86-09,86-10,86-11,86-12,87-01,87-02,87-03,87-04,87-05,87-06,87-07,87-08,87-09,87-10,87-11,87-12,88-01,88-02,88-03,88-04,88-05,88-06,88-07,88-08,88-09,88-10,88-11,88-12,89-01,89-02,89-03,89-04,89-05,89-06,89-07,89-08,89-09,89-10,89-11,89-12,90-01,90-02,90-03,90-04,90-05,90-06,90-07,90-08,90-09,90-10,90-11,90-12,91-01,91-02,91-03,91-04,91-05,91-06,91-07,91-08,91-09,91-10,91-11,91-12,92-01,92-02,92-03,92-04,92-05,92-06,92-07,92-08,92-09,92-10,92-11,92-12,93-01,93-02,93-03,93-04,93-05,93-06,93-07,93-08,93-09,93-10,93-11,93-12,94-01,94-02,94-03,94-04,94-05,94-06,94-07,94-08,94-09,94-10,94-11,94-12,95-01,95-02,95-03,95-04,95-05,95-06,95-07,95-08,95-09,95-10,95-11,95-12,96-01,96-02,96-03,96-04,96-05,96-06,96-07,96-08,96-09,96-10,96-11,96-12,97-01,97-02,97-03,97-04,97-05,97-06,97-07,97-08,97-09,97-10,97-11,97-12,98-01,98-02,98-03,98-04,98-05,98-06,98-07,98-08,98-09,98-10,98-11,98-12,99-01,99-02,99-03,99-04,99-05,99-06,99-07,99-08,99-09,99-10,99-11,99-12,00-01,00-02,00-03,00-04,00-05,00-06,00-07,00-08,00-09,00-10,00-11,00-12,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,01-11,01-12,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,03-11,03-12,04-01,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11,04-12,05-01,05-02,05-03,05-04,05-05,05-06,05-07,05-08,05-09,05-10,05-11,05-12,06-01,06-02,06-03,06-04,06-05,06-06,06-07,06-08,06-09,06-10,06-11,06-12,07-01,07-02,07-03,07-04,07-05,07-06,07-07,07-08,07-09,07-10,07-11,07-12,08-01,08-02,08-03,08-04,08-05,08-06,08-07,08-08,08-09,08-10,08-11,08-12,09-01,09-02,09-03,09-04,09-05,09-06,09-07,09-08,09-09,09-10,09-11,09-12,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,10-11,10-12,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,11-09,11-10,11-11,11-12,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09,12-10,12-11,12-12,13-01,13-02,13-03,13-04,13-05,13-06,13-07,13-08,13-09,13-10,13-11,13-12,14-01,14-02,14-03,14-04,14-05,14-06,14-07,14-08,14-09,14-10,14-11,14-12,15-01,15-02,15-03,15-04,15-05,15-06,15-07,15-08,15-09,15-10,15-11,15-12,16-01,16-02,16-03,16-04,16-05,16-06,16-07,16-08,16-09,16-10,16-11,16-12,17-01,17-02,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,18-01,18-02,18-03,18-04,18-05,18-06,18-07,18-08,18-09,18-10,18-11,18-12,19-01,19-02,19-03,19-04,19-05,19-06,19-07,19-08,19-09,19-10,19-11,19-12,20-01,20-02,20-03,20-04,20-05,20-06,20-07,20-08,20-09,20-10,20-11,20-12,index,adm0,adm1,adm2,intersection_ratio
0,-38.25,83.25,1.68,1.81,-0.19,-0.17,-0.14,-0.84,-0.85,1.55,-0.19,-0.16,0.11,0.00,0.00,-0.12,-0.56,-0.63,-0.54,-1.03,0.22,0.08,0.63,0.60,0.52,0.50,0.49,0.34,1.13,2.33,1.98,1.47,1.30,2.07,1.39,1.18,2.04,1.73,1.47,1.10,0.88,0.69,0.59,2.36,5.06,4.18,4.75,4.03,3.60,3.06,2.60,2.17,1.41,1.05,0.89,1.04,2.08,1.51,3.32,2.82,2.75,0.00,0.00,-0.15,-0.35,-0.48,-0.41,-0.48,-0.29,-1.39,0.22,0.21,0.84,0.00,0.00,-0.15,-0.48,-0.64,-0.54,-0.89,1.22,1.40,3.50,2.97,2.96,2.51,2.14,1.66,1.20,0.78,0.66,1.20,2.71,2.26,3.45,2.93,2.67,2.27,1.93,1.25,0.74,0.65,0.56,2.98,3.10,5.24,4.72,4.01,3.84,3.27,2.78,3.10,2.45,1.73,1.47,3.49,2.86,2.12,2.92,2.48,2.05,1.75,1.49,1.74,3.26,5.38,0.00,-0.57,-0.62,-1.98,-2.27,-2.19,-2.23,0.00,0.00,-0.12,0.40,0.52,0.50,1.00,1.50,1.54,3.54,3.01,2.82,2.40,2.04,1.72,1.39,1.10,0.93,0.85,1.52,0.83,1.57,1.33,1.35,0.00,0.00,0.04,-0.30,-0.41,-0.34,-1.10,0.24,-0.43,-1.37,0.00,-0.09,-0.07,-0.06,0.36,0.21,0.10,0.10,1.82,1.50,2.46,2.44,2.07,2.84,2.42,2.06,1.95,1.34,0.89,0.76,0.44,1.23,2.52,1.85

In [6]:
# The score of an adm2 is the average of all measures weighted by intersection_ratio
value_cols = list(df.filter(regex=r'^\d\d-\d\d$').columns)
df[value_cols] *= df[value_cols].multiply(df['intersection_ratio'], axis=0)

id_vars = ['adm0', 'adm1', 'adm2']

aggfunc = {col: 'mean' for col in value_cols}
aggfunc['intersection_ratio'] = [np.sum, 'count']

pivot = pd.pivot_table(
    df,
    index=id_vars,
    values=value_cols + ['intersection_ratio'],
    aggfunc=aggfunc
)

pivot.columns = ['_'.join(x) if x[0] == 'intersection_ratio' else x[0] for x in pivot.columns]

pivot[value_cols] = pivot[value_cols].divide(pivot['intersection_ratio_sum'], axis=0)

# Reshape to long
melt = pivot.reset_index().melt(
    id_vars=id_vars,
    value_vars=value_cols,
    var_name='time',
    value_name='precipitation'
)

melt = melt.merge(
    pivot[['intersection_ratio_count', 'intersection_ratio_sum']],
    left_on=id_vars,
    right_index=True
)

# Fix year mapping
year_map = {}
month_map = {}
for t in value_cols:
    year, month = t.split('-')
    year = int(year)
    year = 1900 + year if year > 20 else 2000 + year
    year_map[t] = year
    month_map[t] = int(month)
    
melt['year'] = melt['time'].map(year_map)
melt['month'] = melt['time'].map(month_map)
melt.drop(columns='time', inplace=True)

# Save results
melt.to_csv(output_folder / f'{output_folder.name}.csv', index=False)
print(melt.shape)
melt.head()

(10479600, 8)


,adm0,adm1,adm2,precipitation,intersection_ratio_count,intersection_ratio_sum,year,month
0,AFG,AFG.10_1,AFG.10.1_1,0.387072,9,4.110212,1980,1
21300,AFG,AFG.10_1,AFG.10.1_1,0.812244,9,4.110212,1980,2
42600,AFG,AFG.10_1,AFG.10.1_1,1.340727,9,4.110212,1980,3
63900,AFG,AFG.10_1,AFG.10.1_1,1.000916,9,4.110212,1980,4
85200,AFG,AFG.10_1,AFG.10.1_1,0.746590,9,4.110212,1980,5


# Aggregate

In [7]:
df = pd.read_csv(output_folder / f'{output_folder.name}.csv')

time_groups = {'yearly': ['year'], 'monthly': ['year', 'month']}
loc_groups = {'country': ['adm0'], 
              'edo': ['adm0', 'adm1'], 
              'mun': ['adm0', 'adm1', 'adm2']}

groups = {}
for loc_name, loc_group in loc_groups.items():
    for time_name, time_group in time_groups.items():
        groups[f'{loc_name}_{time_name}'] = loc_group + time_group

done = [Path(x).name for x in glob(str(output_folder / 'CRU_*.csv'))]
    
for group_name, group in groups.items():
    fname = f'{output_folder.name}_{group_name}.csv'
    if fname in done:
        continue

    print(group_name)
    collapsed = []

    pivot = pd.pivot_table(df, 
                           values=['precipitation'],
                           index=group,
                           aggfunc=[np.mean, np.median, np.std, 'count'])
    pivot.columns = ['_'.join(x) for x in pivot.columns]
    pivot.to_csv(output_folder / fname)

country_yearly
country_monthly
edo_yearly
edo_monthly
mun_yearly
mun_monthly


# Ethnic

In [4]:
cru = pd.read_csv(output_folder / '../CRU/ethnic_map.csv')
print(df.shape)
df = df.merge(cru, on=['lon', 'lat'])
print(df.shape)
df.head()

(64040, 494)
(11386, 498)


,lon,lat,80-01,80-02,80-03,80-04,80-05,80-06,80-07,80-08,80-09,80-10,80-11,80-12,81-01,81-02,81-03,81-04,81-05,81-06,81-07,81-08,81-09,81-10,81-11,81-12,82-01,82-02,82-03,82-04,82-05,82-06,82-07,82-08,82-09,82-10,82-11,82-12,83-01,83-02,83-03,83-04,83-05,83-06,83-07,83-08,83-09,83-10,83-11,83-12,84-01,84-02,84-03,84-04,84-05,84-06,84-07,84-08,84-09,84-10,84-11,84-12,85-01,85-02,85-03,85-04,85-05,85-06,85-07,85-08,85-09,85-10,85-11,85-12,86-01,86-02,86-03,86-04,86-05,86-06,86-07,86-08,86-09,86-10,86-11,86-12,87-01,87-02,87-03,87-04,87-05,87-06,87-07,87-08,87-09,87-10,87-11,87-12,88-01,88-02,88-03,88-04,88-05,88-06,88-07,88-08,88-09,88-10,88-11,88-12,89-01,89-02,89-03,89-04,89-05,89-06,89-07,89-08,89-09,89-10,89-11,89-12,90-01,90-02,90-03,90-04,90-05,90-06,90-07,90-08,90-09,90-10,90-11,90-12,91-01,91-02,91-03,91-04,91-05,91-06,91-07,91-08,91-09,91-10,91-11,91-12,92-01,92-02,92-03,92-04,92-05,92-06,92-07,92-08,92-09,92-10,92-11,92-12,93-01,93-02,93-03,93-04,93-05,93-06,93-07,93-08,93-09,93-10,93-11,93-12,94-01,94-02,94-03,94-04,94-05,94-06,94-07,94-08,94-09,94-10,94-11,94-12,95-01,95-02,95-03,95-04,95-05,95-06,95-07,95-08,95-09,95-10,95-11,95-12,96-01,96-02,96-03,96-04,96-05,96-06,96-07,96-08,96-09,96-10,96-11,96-12,97-01,97-02,97-03,97-04,97-05,97-06,97-07,97-08,97-09,97-10,97-11,97-12,98-01,98-02,98-03,98-04,98-05,98-06,98-07,98-08,98-09,98-10,98-11,98-12,99-01,99-02,99-03,99-04,99-05,99-06,99-07,99-08,99-09,99-10,99-11,99-12,00-01,00-02,00-03,00-04,00-05,00-06,00-07,00-08,00-09,00-10,00-11,00-12,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,01-11,01-12,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,03-11,03-12,04-01,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11,04-12,05-01,05-02,05-03,05-04,05-05,05-06,05-07,05-08,05-09,05-10,05-11,05-12,06-01,06-02,06-03,06-04,06-05,06-06,06-07,06-08,06-09,06-10,06-11,06-12,07-01,07-02,07-03,07-04,07-05,07-06,07-07,07-08,07-09,07-10,07-11,07-12,08-01,08-02,08-03,08-04,08-05,08-06,08-07,08-08,08-09,08-10,08-11,08-12,09-01,09-02,09-03,09-04,09-05,09-06,09-07,09-08,09-09,09-10,09-11,09-12,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,10-11,10-12,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,11-09,11-10,11-11,11-12,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09,12-10,12-11,12-12,13-01,13-02,13-03,13-04,13-05,13-06,13-07,13-08,13-09,13-10,13-11,13-12,14-01,14-02,14-03,14-04,14-05,14-06,14-07,14-08,14-09,14-10,14-11,14-12,15-01,15-02,15-03,15-04,15-05,15-06,15-07,15-08,15-09,15-10,15-11,15-12,16-01,16-02,16-03,16-04,16-05,16-06,16-07,16-08,16-09,16-10,16-11,16-12,17-01,17-02,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,18-01,18-02,18-03,18-04,18-05,18-06,18-07,18-08,18-09,18-10,18-11,18-12,19-01,19-02,19-03,19-04,19-05,19-06,19-07,19-08,19-09,19-10,19-11,19-12,20-01,20-02,20-03,20-04,20-05,20-06,20-07,20-08,20-09,20-10,20-11,20-12,intersection_ratio,NAME,TRIBE_CODE,GID_0
0,8.75,37.25,-1.31,-1.74,-1.45,-1.10,-0.79,-0.99,-1.24,-1.34,-1.72,-1.66,0.12,0.64,0.75,-0.05,-0.35,-0.52,-0.64,-0.59,-0.59,-0.47,-1.02,-1.51,-2.04,-2.01,0.19,0.29,0.71,1.13,1.51,1.76,1.15,1.39,1.10,1.53,2.49,2.96,-0.32,-0.63,-0.32,-1.05,-1.42,-1.82,-2.06,-2.19,0.28,0.54,0.75,0.49,0.65,1.09,1.27,-0.13,-0.27,-0.34,-0.55,-0.68,0.16,0.77,0.32,2.09,2.35,2.06,2.52,2.16,2.29,-0.05,-0.52,-0.87,-0.88,-1.48,-1.93,-2.72,-2.98,-3.14,-2.63,-2.47,-3.16,-3.59,-2.99,-3.09,0.44,0.67,0.99,1.15,1.36,1.75,1.86,1.49,1.79,1.71,0.02,-0.07,-0.39,-1.15,-1.18,-2.17,-2.70,-3.07,-3.07,-3.59,-4.32,-4.27,-4.12,-4.00,-3.45,-4.02,-4.03,-3.83,-4.29,-4.51,-4.84,-4.17,-4.63,-4.79,-4.40,-3.78,-3.63,-3.15,-3.26,-3.02,-2.17,-2.88,-2.92,-2.44,-1.91,-1.73,-1.57,-0.86,-1.68,-1.89,0.67,1.82,2.25,2.22,2.32,2.41,2.47,2.45,2.05,1.70,1.82,2.01,1.98,1.79,1.36,1.47,1.40,2.02,2.71,3.32,3.84,-0.13,-0.77,-0.88,-0.62,-0.16,-0.35,-0.34,-0.23,-0.60,-0.07,-0.08,-0.44,-0.78,-0.74,-0.85,-1.38,-1.67,-2.07,-1.65,-2.57,-2.40,-2.95,-3.36,-3.3

In [5]:
pivot = raster.weighted_pivot(
    df, 
    weight='intersection_ratio', 
    value_name='precipitation', 
    id_vars=['NAME', 'TRIBE_CODE', 'GID_0']
)
utils.map_year_month(pivot, 'time', 1980, 2020)

# Save results
pivot.to_csv(output_folder / 'SCPDSI.csv', index=False)
print(pivot.shape)
pivot.head()

(481668, 9)


,NAME,TRIBE_CODE,GID_0,time,precipitation,intersection_ratio_count,intersection_ratio_sum,year,month
0,ABABDA,1,EGY,80-01,0.171013,12,9.912794,1980,1
979,ABABDA,1,EGY,80-02,0.178830,12,9.912794,1980,2
1958,ABABDA,1,EGY,80-03,0.261796,12,9.912794,1980,3
2937,ABABDA,1,EGY,80-04,0.441934,12,9.912794,1980,4
3916,ABABDA,1,EGY,80-05,0.557372,12,9.912794,1980,5


In [6]:
# Aggregate
time_groups = {'yearly': ['year'], 'monthly': ['year', 'month']}
loc_groups = {'ethnic': ['NAME', 'TRIBE_CODE', 'GID_0']}

groups = {}
for loc_name, loc_group in loc_groups.items():
    for time_name, time_group in time_groups.items():
        groups[f'{loc_name}_{time_name}'] = loc_group + time_group

output_folder = Path('../Output/Precipitaciones/Raster/SCPDSI_Ethnic/')
output_folder.mkdir(exist_ok=True)
done = [Path(x).name for x in glob(str(output_folder / 'SCPDSI_*.csv'))]
    
utils.aggregate_by_groups(pivot, groups, output_folder, ['precipitation'], done=done, batch=None)

Aggregating: 100%|██████████| 2/2 [00:05<00:00,  2.82s/it]
